In [1]:
"""
Load a messages folder and extract conversations
"""
from minichain.message_handler import MessageDB



messages_folder = "/Users/nielswarncke/Documents/agi/minichain/demo/text-entropy-heatmap/.minichain/messages"

message_db = MessageDB(messages_folder)

len(message_db.conversations)

Set default memory to <minichain.memory.SemanticParagraphMemory object at 0x11e2ba1d0>
No memories found in .minichain/memory


2

In [2]:
[i.as_json() for i in message_db.conversations]

[{'meta': {'timestamp': '2023-10-30T23:28:26', 'agent': 'AGI'},
  'path': ['root', 'c30a3853'],
  'messages': [{'chat': {'content': "You are a smart and friendly AGI.\nYou work as a programmer - or sometimes better as a manager of programmers - and fulfill tasks for the user.\nYou are equipped with a wide range of tools, notably:\n- a memory: you can use it to find relevant code sections and more\n- a task board: you can use it to break down complex tasks into sub tasks and assign them to someone to work on\n- the assign tool: you can use it to assign tasks to copies of yourself or a programmer\n- some tools to work with code: you can use them to implement features, refactor code, write tests, etc.\n\nIf the user asks you something simple, do it directly.\nIf the user asks you something complex about the code, make a plan first:\n- try to find relevant memories\n- try to find relevant code sections using the view tool if needed\n- once you know enough to make a plan, create tasks on th

In [3]:
import json
def get_messages(conversation):
    return [i.as_json()['chat'] for i in conversation.messages]

for i in message_db.conversations:
    print(json.dumps(get_messages(i), indent=4))
    print('-'*120)

[
    {
        "content": "You are a smart and friendly AGI.\nYou work as a programmer - or sometimes better as a manager of programmers - and fulfill tasks for the user.\nYou are equipped with a wide range of tools, notably:\n- a memory: you can use it to find relevant code sections and more\n- a task board: you can use it to break down complex tasks into sub tasks and assign them to someone to work on\n- the assign tool: you can use it to assign tasks to copies of yourself or a programmer\n- some tools to work with code: you can use them to implement features, refactor code, write tests, etc.\n\nIf the user asks you something simple, do it directly.\nIf the user asks you something complex about the code, make a plan first:\n- try to find relevant memories\n- try to find relevant code sections using the view tool if needed\n- once you know enough to make a plan, create tasks on the board\n- when you implement full stack features, you need to plan the API interfaces before you assign 

# Evaluate different formats
- format chat as xml/json/etc
- compare loglikelihood

In [4]:
from typing import Dict, List, Optional, Tuple, Union

def function_call_formatter(function_call: Dict) -> str:
    pass

def message_formatter(message: Dict, function_call_formatter) -> str:
    pass

def functions_formatter(functions: List[Dict], function_call_formatter=None) -> str:
    pass

def conversation_formatter(messages: Dict, functions: List, message_formatter, functions_formatter, function_call_formatter) -> str:
    """Adds the function definitions to the system message and formats the chat using the specified formatters"""
    system_prompt_addition = "\n\nYou have access to the following functions:\n"
    for function in functions:
        system_prompt_addition += f"{functions_formatter(function, function_call_formatter)}\n"
    system_prompt_addition += "\nRespond in the following syntax:\n"
    system_prompt_addition += "<function_call> ...arguments </function_call>\n"
    
    text = ""
    for message in messages[:1]:
        if message['role'] == 'system':
            message = message.copy()
            message['content'] += system_prompt_addition
            system_prompt_addition = ""
        text += message_formatter(message, function_call_formatter)
    return text
    

    

In [5]:
from minichain.functions import tool
from pydantic import Field


@tool()
def add(x: int = Field(..., description="First number"), y: int = Field(..., description="Second number")) -> int:
    """Returns the sum of two integers"""
    return x + y


@tool()
def send_message(message: str = Field(..., description="Message text")) -> str:
    """Sends a message to the user"""
    return message

In [17]:
import xml.etree.ElementTree as ET
import json


def dict_to_xml(tag, d):
    """
    Convert a dictionary or list to an XML string.
    Args:
    - tag: the root tag name
    - d: the dictionary or list to convert
    """
    elem = ET.Element(tag)
    if isinstance(d, dict):
        for key, val in d.items():
            child = dict_to_xml(key, val)
            elem.append(child)
    elif isinstance(d, list):
        for item in d:
            child = dict_to_xml('item', item)
            elem.append(child)
    else:
        elem.text = str(d)
    return elem

def convert_to_xml(data):
    """
    Convert a JSON-like structure to an XML string.
    Args:
    - data: the JSON-like structure (dict or list)
    """
    xml_elem = dict_to_xml('function_call', data)
    # convert to string with newline pretty-printing
    ET.indent(xml_elem)
    return ET.tostring(xml_elem, encoding='unicode', method='xml', short_empty_elements=False)

def function_call_to_xml(function_call: Dict) -> str:
    return convert_to_xml(function_call)

def function_call_to_json(function_call: Dict, function_call_formatter=None) -> str:
    return json.dumps(function_call, indent=4)

def message_to_chatml(message: Dict, function_call_formatter) -> str:
    content = message['content']
    role = message['role']
    if message.get('function_call') is not None:
        content += function_call_formatter(message['function_call'])
    if message.get('name') is not None:
        role += f" {message['name']}"
    
    text = f"""<|im_start|>{role}
{content}<|im_end|>
"""
    return text

def functions_to_openapi(function: Dict, function_call_formatter) -> str:
    data = {
        "name": function.name,
        "arguments": function.pydantic_model.model_json_schema(),
    }
    return json.dumps(data, indent=4)


def openapi_to_example(openapi: Dict) -> str:
    breakpoint()
    arguments = {}
    for arg_name, arg in openapi['arguments']['properties'].items():
        # If arg is a reference, get the actual schema
        if arg.get('$ref') is not None:
            arg = openapi['arguments']['definitions'][arg['$ref'].split('/')[-1]]
        # If the arg type is an array, get the schema of the items
        if arg['type'] == 'array':
            arg = [openapi_to_example({'arguments': {'properties': {'item': arg['items']}}})]
            continue
        # If arg is an object, get the schema of the properties
        if arg['type'] == 'object':
            arg = openapi_to_example({'arguments': arg})
            continue
        
        arguments[arg_name] = f"(type: {arg['type']}) <{arg['description']}>"
    function_call = {
        "name": openapi['name'],
        "arguments": arguments
    }
    return function_call

def functions_to_example(function: Dict, function_call_formatter) -> str:
    """Generate an example function call with values replaced by type + description infos"""
    function_call = openapi_to_example(function.pydantic_model.model_json_schema())
    return function_call_formatter(function_call)


example_text = conversation_formatter(
    get_messages(message_db.conversations[0]),
    [add, send_message],
    message_to_chatml,
    functions_to_example, 
    # functions_to_openapi,
    function_call_to_xml
)
print(example_text)

KeyError: 'arguments'

In [11]:
example_text = conversation_formatter(
    get_messages(message_db.conversations[0]),
    [],
    message_to_chatml,
    functions_to_example, 
    function_call_to_json
)
print(example_text)

<|im_start|>system
You are a smart and friendly AGI.
You work as a programmer - or sometimes better as a manager of programmers - and fulfill tasks for the user.
You are equipped with a wide range of tools, notably:
- a memory: you can use it to find relevant code sections and more
- a task board: you can use it to break down complex tasks into sub tasks and assign them to someone to work on
- the assign tool: you can use it to assign tasks to copies of yourself or a programmer
- some tools to work with code: you can use them to implement features, refactor code, write tests, etc.

If the user asks you something simple, do it directly.
If the user asks you something complex about the code, make a plan first:
- try to find relevant memories
- try to find relevant code sections using the view tool if needed
- once you know enough to make a plan, create tasks on the board
- when you implement full stack features, you need to plan the API interfaces before you assign the task to the progra